### Project 02: Create a GUI Notebook Program

In [ ]:
Bao Nguyen
INST326

enter your name and date here

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [ ]:
import tkinter as tk
from tkinter import ttk
import tkinter.filedialog
import datetime
import json

class MainWindow(tk.Tk):
    def __init__(self):
        super().__init__()
        self.geometry("600x400")
        self.title('Notebook and Snippet Manager')
        self.notebook = []
        self.snippets = []

        # Buttons for new note, open, and save
        new_note_button = ttk.Button(self, text="New Note", command=self.new_note)
        new_note_button.pack(side=tk.LEFT, padx=5, pady=5)

        # Buttons for new snippet, save, and load
        new_snippet_button = ttk.Button(self, text="New Snippet", command=self.new_snippet)
        new_snippet_button.pack(side=tk.LEFT, padx=5, pady=5)

        save_snippet_button = ttk.Button(self, text="Save Snippets", command=self.save_snippets_dialog)
        save_snippet_button.pack(side=tk.LEFT, padx=5, pady=5)

        load_snippet_button = ttk.Button(self, text="Load Snippets", command=self.load_snippets_dialog)
        load_snippet_button.pack(side=tk.LEFT, padx=5, pady=5)

    def new_note(self):
        note_window = NoteForm(self, self.notebook)
        note_window.grab_set()

    def new_snippet(self):
        snippet_window = SnippetForm(self, self.snippets)
        snippet_window.grab_set()

    def open_notebook_dialog(self):
        filename = tkinter.filedialog.askopenfilename(title="Open Notebook", filetypes=[("JSON files", "*.json")])
        if filename:
            self.load_notes(filename)

    def open_snippet_dialog(self):
        filename = tkinter.filedialog.askopenfilename(title="Open Snippets", filetypes=[("JSON files", "*.json")])
        if filename:
            self.load_snippets(filename)

    def save_notebook_dialog(self):
        filename = tkinter.filedialog.asksaveasfilename(title="Save Notebook", defaultextension=".json", filetypes=[("JSON files", "*.json")])
        if filename:
            self.save_notes(filename)

    def save_snippets_dialog(self):
        filename = tkinter.filedialog.asksaveasfilename(title="Save Snippets", defaultextension=".json", filetypes=[("JSON files", "*.json")])
        if filename:
            self.save_snippets(filename)

    def load_notes(self, filename):
        try:
            with open(filename, 'r') as file:
                self.notebook = json.load(file, object_hook=lambda d: MakeNote(**d))
            print("Notebook loaded successfully.")
        except FileNotFoundError:
            print(f"Error: Notebook '{filename}' not found.")
        except json.JSONDecodeError:
            print("Error: Failed to decode JSON.")

    def load_snippets(self, filename):
        try:
            with open(filename, 'r') as file:
                self.snippets = json.load(file, object_hook=lambda d: CodeSnippet(**d))
            print("Snippets loaded successfully.")
        except FileNotFoundError:
            print(f"Error: Snippets file '{filename}' not found.")
        except json.JSONDecodeError:
            print("Error: Failed to decode JSON.")

    def save_notes(self, filename):
        try:
            with open(filename, 'w') as file:
                json.dump(self.notebook, file, indent=4, default=lambda obj: obj.__dict__)
            print("Notebook saved successfully.")
        except Exception as e:
            print(f"Error saving notebook: {e}")

    def save_snippets(self, filename):
        try:
            with open(filename, 'w') as file:
                json.dump(self.snippets, file, indent=4, default=lambda obj: obj.__dict__)
            print("Snippets saved successfully.")
        except Exception as e:
            print(f"Error saving snippets: {e}")

    def display_notes(self):
        print("Notes:")
        for note in self.notebook:
            print(f"Title: {note.title}")
            print(f"Text: {note.text}")
            print(f"Link: {note.link}")
            print(f"Tags: {note.tags}")
            print(f"Date Created: {note.date_created}")
            print()

    def display_snippets(self):
        print("Snippets:")
        for snippet in self.snippets:
            print(f"Title: {snippet.title}")
            print(f"Code: {snippet.code}")
            print(f"Description: {snippet.description}")
            print(f"Tags: {snippet.tags}")
            print(f"Date Created: {snippet.date_created}")
            print()
            
class NoteForm(tk.Toplevel):
    def __init__(self, master, notebook):
        super().__init__(master)
        self.title("New Note")
        self.notebook = notebook

        # Note attributes form
        self.title_label = ttk.Label(self, text="Title:")
        self.title_label.grid(row=0, column=0, padx=5, pady=5)
        self.title_entry = ttk.Entry(self)
        self.title_entry.grid(row=0, column=1, padx=5, pady=5)

        self.text_label = ttk.Label(self, text="Text:")
        self.text_label.grid(row=1, column=0, padx=5, pady=5)
        self.text_entry = ttk.Entry(self)
        self.text_entry.grid(row=1, column=1, padx=5, pady=5)

        self.link_label = ttk.Label(self, text="Link:")
        self.link_label.grid(row=2, column=0, padx=5, pady=5)
        self.link_entry = ttk.Entry(self)
        self.link_entry.grid(row=2, column=1, padx=5, pady=5)

        self.tags_label = ttk.Label(self, text="Tags:")
        self.tags_label.grid(row=3, column=0, padx=5, pady=5)
        self.tags_entry = ttk.Entry(self)
        self.tags_entry.grid(row=3, column=1, padx=5, pady=5)

        self.submit_button = ttk.Button(self, text="Submit", command=self.submit)
        self.submit_button.grid(row=4, columnspan=2, padx=5, pady=5)
        
    def submit(self):
        try:
            note_dict = {
                "title": self.title_entry.get(),
                "text": self.text_entry.get(),
                "link": self.link_entry.get(),
                "tags": self.tags_entry.get(),
                "date_created": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            new_note = MakeNote(**note_dict)
            self.notebook.append(new_note)
            self.master.display_notes()  # Update the displayed notes
            self.destroy()
        except Exception as e:
            print(f"Error submitting note: {e}")

class SnippetForm(tk.Toplevel):
    def __init__(self, master, snippets):
        super().__init__(master)
        self.title("New Snippet")
        self.snippets = snippets

        # Snippet attributes form
        self.title_label = ttk.Label(self, text="Title:")
        self.title_label.grid(row=0, column=0, padx=5, pady=5)
        self.title_entry = ttk.Entry(self)
        self.title_entry.grid(row=0, column=1, padx=5, pady=5)

        self.code_label = ttk.Label(self, text="Code:")
        self.code_label.grid(row=1, column=0, padx=5, pady=5)
        self.code_text = tk.Text(self, height=10, width=50)
        self.code_text.grid(row=1, column=1, padx=5, pady=5)

        self.description_label = ttk.Label(self, text="Description:")
        self.description_label.grid(row=2, column=0, padx=5, pady=5)
        self.description_entry = ttk.Entry(self)
        self.description_entry.grid(row=2, column=1, padx=5, pady=5)

        self.tags_label = ttk.Label(self, text="Tags (comma separated):")
        self.tags_label.grid(row=3, column=0, padx=5, pady=5)
        self.tags_entry = ttk.Entry(self)
        self.tags_entry.grid(row=3, column=1, padx=5, pady=5)

        self.submit_button = ttk.Button(self, text="Submit", command=self.submit)
        self.submit_button.grid(row=4, columnspan=2, padx=5, pady=5)

    def submit(self):
        tags = self.tags_entry.get().split(',')
        snippet = CodeSnippet(
            title=self.title_entry.get(),
            code=self.code_text.get("1.0", "end-1c"),
            description=self.description_entry.get(),
            tags=tags
        )
        self.snippets.append(snippet)
        self.master.display_snippets()  # Display updated list of snippets
        self.destroy()

class MakeNote:
    def __init__(self, title, text, link, tags, date_created):
        self.title = title
        self.text = text
        self.link = link
        self.tags = tags
        self.date_created = date_created

class CodeSnippet:
    def __init__(self, title, code, description="", tags=None, date_created=None):
        self.title = title
        self.code = code
        self.description = description
        self.tags = tags if tags else []
        self.date_created = date_created if date_created else datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

if __name__ == '__main__':
    main_window = MainWindow()
    main_window.mainloop()


#### Print your three notes below

In [ ]:
# print your notes here